In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_717139/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_15_0,0.999760,0.829056,0.779760,0.995952,0.001041,1.143104,0.787785,5.131553e-03,0.066592,0.032258,1.015381,0.032766,143.735973,285.071146,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_45_0,0.999687,0.829283,0.779816,0.998982,0.001355,1.141588,0.787585,3.431236e-03,0.072719,0.036809,1.020027,0.037388,143.208081,284.543253,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_26_7,0.999687,0.829283,0.779816,1.000000,0.001355,1.141588,0.787585,2.855509e-09,0.072719,0.036809,1.020027,0.037388,143.208081,284.543253,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_19_7,0.999687,0.829283,0.779816,0.999048,0.001355,1.141588,0.787585,2.520944e-03,0.072719,0.036809,1.020027,0.037388,143.208081,284.543253,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_19_6,0.999687,0.829283,0.779816,0.999048,0.001355,1.141588,0.787585,2.520944e-03,0.072719,0.036809,1.020027,0.037388,143.208081,284.543253,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,model_1_1_4,0.929379,0.773815,0.735715,0.793635,0.305770,1.512499,0.945331,4.759741e-01,1.004678,0.552965,5.519730,0.561674,132.369842,273.705015,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
485,model_1_1_3,0.928442,0.772993,0.736233,0.792569,0.309830,1.517995,0.943481,4.784327e-01,1.008897,0.556624,5.579743,0.565390,132.343461,273.678634,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
486,model_1_1_2,0.927827,0.771196,0.738139,0.795334,0.312492,1.530014,0.936662,4.720563e-01,1.001727,0.559010,5.619083,0.567813,132.326354,273.661527,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
487,model_1_1_1,0.926739,0.769927,0.738157,0.794045,0.317203,1.538499,0.936597,4.750294e-01,0.999707,0.563208,5.688719,0.572078,132.296428,273.631600,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
